<a href="https://colab.research.google.com/github/jhenningsen/Equity_Analysis/blob/main/LangStudio/SMA_Analysis_II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pandas_ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.3/240.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 69.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 77.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 MB 13.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.43.0
    Uninstalling llvmlite-0.43.0:
      Successfully uninstalled llvmlite-0.43.0
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninsta

In [6]:
import pandas as pd
import yfinance as yf
import pandas_ta as ta
import numpy as np

# 1. Setup Ranges
SMA_RANGE = range(3, 21)  # 3 to 20 inclusive
BB_PERIODS = [10, 20, 30]
SYMBOLS = ["TSLA", "NVDA", "AAPL", "MSFT", "AMD", "META", "GOOGL", "AMZN", "NFLX"]

# 2. Optimized Data Cache
print("Fetching fresh data...")
data_cache = {}
for s in SYMBOLS:
    df = yf.download(s, period="5y", interval="1d", progress=False, auto_adjust=True)
    if not df.empty:
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = df.columns.get_level_values(0)
        data_cache[s] = df

# 3. Execution
results = []

print("Analyzing Parameter Surface...")
for sma_val in SMA_RANGE:
    for bb_val in BB_PERIODS:
        trade_data = []

        for symbol, df_orig in data_cache.items():
            df = df_orig.copy()

            # Indicators
            df['SMA'] = ta.sma(df['Close'], length=sma_val)
            bb = ta.bbands(df['Close'], length=bb_val, std=2)
            if bb is None: continue

            bbm_col = [c for c in bb.columns if c.startswith('BBM')][0]
            df['Mid_B'] = bb[bbm_col]

            # Logic: Cross above SMA AND Below BB Midpoint
            df['Prev_Close'] = df['Close'].shift(1)
            df['Prev_SMA'] = df['SMA'].shift(1)

            cond_cross = (df['Close'] > df['SMA']) & (df['Prev_Close'] <= df['Prev_SMA'])
            cond_value = df['Close'] < df['Mid_B']
            df['Signal'] = np.where(cond_cross & cond_value, 1, 0)

            # Calculate Multi-Window Forward Returns
            df['Ret_3D'] = df['Close'].pct_change(3).shift(-3)
            df['Ret_5D'] = df['Close'].pct_change(5).shift(-5)
            df['Ret_10D'] = df['Close'].pct_change(10).shift(-10)

            # Filter for trade outcomes
            trades = df[df['Signal'] == 1][['Ret_3D', 'Ret_5D', 'Ret_10D']]
            trade_data.append(trades)

        # Aggregate all trades for this parameter set
        all_trades = pd.concat(trade_data)
        if not all_trades.empty:
            results.append({
                "SMA": sma_val,
                "BB": bb_val,
                "Total_Trades": len(all_trades),
                # ADD THESE THREE LINES:
                "Wins_3D": (all_trades['Ret_3D'] > 0).sum(),
                "Wins_5D": (all_trades['Ret_5D'] > 0).sum(),
                "Wins_10D": (all_trades['Ret_10D'] > 0).sum(),

                "Avg_3D": all_trades['Ret_3D'].mean(),
                "Avg_5D": all_trades['Ret_5D'].mean(),
                "Avg_10D": all_trades['Ret_10D'].mean()
            })

# 4. Visualization
master_results = pd.DataFrame(results)

# Pivot for the 5-day return as the primary benchmark
pivot_5d = master_results.pivot(index="SMA", columns="BB", values="Avg_5D")

print("\n--- Heatmap: Average 5-Day Return by Parameters ---")
display(pivot_5d.style.background_gradient(cmap='RdYlGn', axis=None).format("{:.2%}"))

# Pivot for the 10-day return as the primary benchmark
pivot_10d = master_results.pivot(index="SMA", columns="BB", values="Avg_10D")

print("\n--- Heatmap: Average 10-Day Return by Parameters ---")
display(pivot_10d.style.background_gradient(cmap='RdYlGn', axis=None).format("{:.2%}"))

print("\n--- Summary: Top 5 Parameter Sets (by 10-Day Return) ---")
display(master_results.sort_values('Avg_10D', ascending=False).head(5))

Fetching fresh data...
Analyzing Parameter Surface...

--- Heatmap: Average 5-Day Return by Parameters ---


BB,10,20,30
SMA,,,
3,0.02%,0.21%,0.20%
4,-0.38%,-0.08%,-0.03%
5,-0.48%,-0.14%,0.02%
6,-0.77%,-0.28%,0.09%
7,-1.09%,-0.52%,-0.17%
8,-0.99%,-0.36%,0.09%
9,-0.48%,-0.14%,0.20%
10,nan%,-0.07%,0.10%
11,0.48%,0.14%,0.31%



--- Heatmap: Average 10-Day Return by Parameters ---


BB,10,20,30
SMA,,,
3,0.64%,0.83%,0.85%
4,0.17%,0.50%,0.51%
5,0.02%,0.37%,0.51%
6,-0.22%,0.32%,0.70%
7,-0.22%,0.42%,0.70%
8,-0.20%,0.46%,0.92%
9,0.43%,0.79%,1.09%
10,nan%,0.79%,0.97%
11,1.92%,0.88%,1.06%



--- Summary: Top 5 Parameter Sets (by 10-Day Return) ---


,SMA,BB,Total_Trades,Wins_3D,Wins_5D,Wins_10D,Avg_3D,Avg_5D,Avg_10D
26,12,10,58,35,33,42,0.010968,0.008602,0.032425
32,14,10,79,45,48,55,0.009717,0.007358,0.022664
23,11,10,44,23,21,30,0.008303,0.004804,0.019221
35,15,10,87,44,48,56,0.003409,0.001308,0.017934
29,13,10,73,43,42,48,0.010536,0.006216,0.017470
